In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import VarianceThreshold

In [3]:
t = np.genfromtxt('SUSY.csv', delimiter=',', usecols=(9,10,11,12,13,14,15,16,17,18), skip_footer=4000)
v = np.genfromtxt('SUSY.csv', delimiter=',', usecols=(0), skip_footer=4000)

KeyboardInterrupt: 

In [ ]:
sel = VarianceThreshold(threshold=(.9 * (1 - .9)))
dataset_susy = sel.fit_transform(t)
dataset_susy[0,:]
t_treino, t_teste, v_treino, v_teste = train_test_split(dataset_susy, v, test_size=0.3, random_state=0)

for i in range(1, 19):
    knn = KNeighborsClassifier(n_neighbors=i, p=2)
    knn.fit(t_treino, v_treino)
    labels = knn.predict(t_teste)
    print("{} : {}".format(i, knn.score(t_teste, v_teste)))